In [15]:
# Regularized part 2

In [16]:
import pandas as pd
import numpy as np
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.metrics import *
import os

In [17]:
df = pd.read_csv(r"/home/sarthakredasani/Documents/CDAC_ML/Datasets/Datasets/Housing.csv")
df.head(5)

,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,42000.0,5850,3,1,2,yes,no,yes,no,no,1,no
1,38500.0,4000,2,1,1,yes,no,no,no,no,0,no
2,49500.0,3060,3,1,1,yes,no,no,no,no,0,no
3,60500.0,6650,3,1,2,yes,yes,no,no,no,0,no
4,61000.0,6360,2,1,1,yes,no,no,no,no,0,no


In [18]:
# target colunm is price
# let's split the data

In [19]:
# first let's do onehotencoding using pandas

dum_df = pd.get_dummies(df, drop_first=True)
X, y = dum_df.drop('price', axis =1), dum_df['price']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25)

In [21]:
# let's do LR
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2_score(y_test, y_pred)

0.6355877562459125

In [22]:
# let's do Ridge

alphas = [0.001, 0.01, 0.1, 0.5, 1, 2]
scores = []
for i in alphas:
    ridge = Ridge(alpha=i)
    ridge.fit(X_train, y_train)
    y_pred = ridge.predict(X_test)
    scores.append([i, r2_score(y_test, y_pred)])


In [23]:
# let's make the Dataframe

df_scores = pd.DataFrame(scores, columns=['alphas', 'score'])
df_scores.sort_values('score', ascending=False).head(5)


,alphas,score
5,2.00,0.637691
4,1.00,0.636779
3,0.50,0.636222
2,0.10,0.635721
1,0.01,0.635601


In [26]:
# let's do Lasso

alphas = [0.001, 0.01, 0.1, 0.5, 1, 2]

scores1 = []
for i in alphas:
    lasso = Lasso(alpha=i)
    lasso.fit(X_train, y_train)
    y_pred = lasso.predict(X_test)
    scores1.append([i, r2_score(y_test, y_pred)])

In [27]:
df_scores = pd.DataFrame(scores1, columns=['alpha', 'score'])
df_scores.sort_values('score', ascending=False).head(1)

,alpha,score
5,2.0,0.635607


In [28]:
# now let's do ElasticNet 

In [29]:
ratios = [0.01, 0.25, 0.5, 0.75, 0.9]
alphas = [0.001, 0.01, 0.1, 0.5, 1, 2]
scores = []
for a in alphas:
    for r in ratios:
        el = ElasticNet(alpha=a, l1_ratio=r)
        el.fit(X_train, y_train)
        y_pred = el.predict(X_test)
        scores.append([r, a, r2_score(y_test, y_pred)])

In [31]:
df_scores = pd.DataFrame(scores, columns=['ratio', 'alpha', 'score'])
df_scores.sort_values('score', ascending=False).head()

,ratio,alpha,score
13,0.75,0.10,0.639500
14,0.90,0.10,0.638804
5,0.01,0.01,0.638787
6,0.25,0.01,0.638297
7,0.50,0.01,0.637618


In [33]:
# fit the best model on whole data
# in the above cell the first entry is the best

In [34]:
best_model = ElasticNet(alpha=0.1, l1_ratio=0.75)
best_model.fit(X, y)

ElasticNet(alpha=0.1, l1_ratio=0.75)

In [35]:
# Inferencing


In [36]:
tst = pd.read_csv(r"/home/sarthakredasani/Documents/CDAC_ML/Datasets/Datasets/tst_Housing.csv")
dum_tst = pd.get_dummies(tst, drop_first=True)
dum_tst.columns

Index(['lotsize', 'bedrooms', 'bathrms', 'stories', 'garagepl', 'recroom_yes',
       'fullbase_yes'],
      dtype='object')

In [37]:
tst

,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,5600,3,1,2,yes,no,yes,no,no,1,no
1,4355,2,1,1,yes,no,no,no,no,0,no
2,3412,3,1,1,yes,no,no,no,no,0,no
3,6666,2,2,2,yes,yes,no,no,no,0,no


In [38]:
X.columns

Index(['lotsize', 'bedrooms', 'bathrms', 'stories', 'garagepl', 'driveway_yes',
       'recroom_yes', 'fullbase_yes', 'gashw_yes', 'airco_yes',
       'prefarea_yes'],
      dtype='object')

In [41]:
# adding the colunms to dum_tst which are not in X_colunms with default values

In [43]:
dum_tst['driveway_yes']=False
dum_tst['gashw_yes']=False
dum_tst['airco_yes']=False
dum_tst['prefarea_yes']=False
dum_tst


,lotsize,bedrooms,bathrms,stories,garagepl,recroom_yes,fullbase_yes,driveway_yes,gashw_yes,airco_yes,prefarea_yes
0,5600,3,1,2,1,False,True,False,False,False,False
1,4355,2,1,1,0,False,False,False,False,False,False
2,3412,3,1,1,0,False,False,False,False,False,False
3,6666,2,2,2,0,True,False,False,False,False,False


In [42]:
# Setting the order of dum_tst same as of that X_colunms 
#  so that it will be easy to predict the outconmes

In [45]:
dum_tst = dum_tst[list(X.columns)]
dum_tst.columns

Index(['lotsize', 'bedrooms', 'bathrms', 'stories', 'garagepl', 'driveway_yes',
       'recroom_yes', 'fullbase_yes', 'gashw_yes', 'airco_yes',
       'prefarea_yes'],
      dtype='object')

In [46]:
best_model.predict(dum_tst)

array([60847.88984178, 37590.30668148, 36196.75847424, 70370.32590499])

In [47]:
best_model.coef_

array([3.73672612e+00, 2.13018452e+03, 1.30844794e+04, 6.75104738e+03,
       4.36856640e+03, 5.61781678e+03, 4.30891838e+03, 5.35556084e+03,
       7.86038213e+03, 1.11186122e+04, 8.27783790e+03])

In [48]:
X.columns

Index(['lotsize', 'bedrooms', 'bathrms', 'stories', 'garagepl', 'driveway_yes',
       'recroom_yes', 'fullbase_yes', 'gashw_yes', 'airco_yes',
       'prefarea_yes'],
      dtype='object')

In [51]:
features = pd.DataFrame({'columns' : list(X.columns), 'coef': list(best_model.coef_)})
features

,columns,coef
0,lotsize,3.736726
1,bedrooms,2130.184523
2,bathrms,13084.479406
3,stories,6751.047380
4,garagepl,4368.566396
5,driveway_yes,5617.816779
6,recroom_yes,4308.918376
7,fullbase_yes,5355.560842
8,gashw_yes,7860.382133
9,airco_yes,11118.612248
